In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pytorch_lightning as pl
import ml
import experiments
import torch
from shared.constants import TMP_PATH

/data/pella/projects/University/Thesis/Thesis/code/env/lib/python3.9/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


[2022-02-14 23:36:01,991][simple_parsing.helpers.serialization.serializable][DEBUG] Registering a new Serializable subclass: <class 'simple_parsing.helpers.serialization.serializable.Serializable'>
[2022-02-14 23:36:01,992][simple_parsing.helpers.serialization.serializable][DEBUG] parents: [<class 'simple_parsing.helpers.serialization.serializable.SerializableMixin'>]
[2022-02-14 23:36:01,993][simple_parsing.helpers.serialization.serializable][DEBUG] Registering a new Serializable subclass: <class 'simple_parsing.helpers.serialization.serializable.FrozenSerializable'>
[2022-02-14 23:36:01,994][simple_parsing.helpers.serialization.serializable][DEBUG] parents: [<class 'simple_parsing.helpers.serialization.serializable.SerializableMixin'>]
[2022-02-14 23:36:01,995][simple_parsing.helpers.serialization.serializable][DEBUG] Registering a new Serializable subclass: <class 'simple_parsing.helpers.serialization.serializable.SimpleSerializable'>
[2022-02-14 23:36:01,997][simple_parsing.helpers

In [3]:
dataset = ml.StarWars()
data = dataset[0]
data

[2022-02-14 23:36:02,275][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetSchema'>, drop extra fields: True
[2022-02-14 23:36:02,276][simple_parsing.helpers.serialization.decoding][DEBUG] name = name, field_type = <class 'str'>
[2022-02-14 23:36:02,277][simple_parsing.helpers.serialization.decoding][DEBUG] name = database, field_type = <class 'str'>
[2022-02-14 23:36:02,277][simple_parsing.helpers.serialization.decoding][DEBUG] name = description, field_type = <class 'str'>
[2022-02-14 23:36:02,278][simple_parsing.helpers.serialization.decoding][DEBUG] name = versions, field_type = typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-14 23:36:02,279][simple_parsing.helpers.serialization.decoding][DEBUG] Decoding a Dict field: typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-14 23:36:02,279][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetVe

HeteroData(
  Character={ x=[113, 32] },
  (Character, INTERACTIONS, Character)={
    edge_attr=[958, 0],
    edge_index=[2, 958],
    timestamp=[958]
  },
  (Character, MENTIONS, Character)={
    edge_attr=[1120, 0],
    edge_index=[2, 1120],
    timestamp=[1120]
  }
)

In [4]:
node_type = 'Character'
repr_dim = 32
save_path = TMP_PATH.joinpath('pyg-sage-comopt')
data_module = ml.EdgeLoaderDataModule(
    data,
    batch_size=16, num_samples=[4] * 2,
    num_workers=0, node_type=node_type, neg_sample_ratio=1
)

In [5]:
embedding_module = experiments.GraphSAGEModule(node_type, data.metadata(), repr_dim, n_layers=2)
prediction_module = experiments.LinkPredictionModule(embedding_module, dist='cosine')

In [6]:
pretrain_model = experiments.LinkPredictionNet(prediction_module)

AttributeError: module 'experiments' has no attribute 'LinkPredictionNet'

In [ ]:
trainer = pl.Trainer(
    # gpus=1,
    callbacks=[
        pl.callbacks.ModelSummary(),
        pl.callbacks.LearningRateMonitor(),
        pl.callbacks.EarlyStopping(monitor="val/loss", min_delta=0.00, patience=5, verbose=True, mode="min")
    ],
    max_epochs=20,
    enable_model_summary=True,
    default_root_dir=str(save_path),
    # logger=wandb_logger
)
trainer.fit(pretrain_model, data_module)

In [ ]:
initializer = experiments.KMeansInitializer(repr_dim=32, k=5, niter=20, verbose=True, nredo=5)
assignment, centroids = initializer.fit(trainer, embedding_module, data_module)

In [ ]:
clustering_module = experiments.ExplicitClusteringModule(
    repr_dim=repr_dim, n_clusters=5, centroids=torch.tensor(centroids), dist='cosine'
)

In [ ]:
model = experiments.SelfSupervisedClusteringNet(
    prediction_module, clustering_module
)

In [ ]:
trainer = pl.Trainer(
    # gpus=1,
    callbacks=[
        pl.callbacks.ModelSummary(),
        pl.callbacks.LearningRateMonitor(),
        pl.callbacks.EarlyStopping(monitor="val/loss", min_delta=0.00, patience=5, verbose=True, mode="min")
    ],
    max_epochs=20,
    enable_model_summary=True,
    default_root_dir=str(save_path),
    # logger=wandb_logger
)
trainer.fit(model, data_module)

In [ ]:
predictions = trainer.predict(model, data_module)
embeddings, assignments = map(lambda x: torch.cat(x, dim=0).detach().cpu(), zip(*predictions))
assignments = torch.argmax(assignments, dim=1)

In [ ]:
from shared.constants import BENCHMARKS_RESULTS

save_path = BENCHMARKS_RESULTS.joinpath('analysis', 'pyg-sage-comopt')
save_path.mkdir(parents=True, exist_ok=True)

In [ ]:
from shared.graph import CommunityAssignment
import pandas as pd

labeling = pd.Series(assignments.squeeze(), index=dataset.node_mapping(), name="cid")
labeling.index.name = "nid"
comlist = CommunityAssignment(labeling)
comlist.save_comlist(save_path.joinpath('schema.comlist'))

In [ ]:
from datasets.scripts import export_to_visualization

export_to_visualization.run(
    export_to_visualization.Args(
        dataset='star-wars',
        version='base',
        run_paths=[str(save_path)]
    )
)

In [ ]:
from shared.graph import DataGraph
from shared.schema import DatasetSchema, GraphSchema

# Calculate Evaluation Metrics
DATASET = DatasetSchema.load_schema('star-wars')
schema = GraphSchema.from_dataset(DATASET)
G = DataGraph.from_schema(schema)
G.to_undirected()

In [ ]:
from benchmarks.evaluation import get_metric_list

metrics = get_metric_list(ground_truth=False, overlapping=False)

results = pd.DataFrame([
    {
        'metric': metric_cls.metric_name(),
        'value': metric_cls.calculate(G, comlist)
    }
    for metric_cls in metrics]
)
results